### Extract apply link domains

The following code extract the domain of apply links. The extraction is used to decide which apply boards are mostly used. I'm aiming to automate the application process at these job boards.

__DON'T REMOVE__

In [ ]:
import pandas as pd
from src.db import DB
import re

db = DB("results/jobs.sqlite")
# https://something.domain.com/blahblahblah -> domain.com
pat = re.compile(r"https?:\/\/(.*\.)?(.*\..*?)\/.*")
q = """
SELECT d.id, d.apply_link, c.name FROM details as d 
LEFT JOIN company as c ON d.company_id = c.id
WHERE apply_link IS NOT NULL;
"""
db.cursor.execute(q)
raw = db.cursor.fetchall()
df = pd.DataFrame(columns=["id","apply_link","company_name"],data=raw)
df["apply_board"] = df["apply_link"].apply(lambda lnk: pat.findall(lnk)[0][-1])
df['apply_board'].value_counts()

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from time import sleep
# from selenium.webdriver.support import expected_conditions as ec
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
driver.driver.refresh()
sleep(2)
form = driver.driver.find_element("xpath",)
scanner = Scanner(driver.driver,"//div[contains(@data-automation-id,'Page')]")
scanner.run()

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from src.routines.myworkday import Routine
from time import sleep
# from selenium.webdriver.support import expected_conditions as ec
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
# driver.driver.implicitly_wait(10)
# routine = Routine(driver.driver,"https://td.wd3.myworkdayjobs.com/en-US/TD_Bank_Careers/job/Toronto%2C-Ontario/Sr-Software-Engineer--ETrading--ION_R_1346079/apply/applyManually")
# routine.goto_url().pause(2).get_creds().sign_in().pause(3).refresh().pause(5).my_information().pause(1).find_click_submit_btn().pause(5).my_experience()

In [5]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from selenium.webdriver import Keys, ActionChains
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
scanner = Scanner(driver.driver,"div[contains(@data-automation-id,'Page')]")
scanner.run()
# chain = ActionChains(driver)
# form = driver.find_element("xpath","//div[contains(@data-automation-id,'Page')]")
# el = form.find_element("xpath","preceding-sibling::*")

input multiselect_input
input radio
button dropdown
input text
input text
input checkbox
input text
input text
button dropdown
input text
button dropdown
input multiselect_input
input text
input text


In [6]:
el = driver.driver.switch_to.active_element

In [7]:
el.click()